<a href="https://colab.research.google.com/github/dit21dsc/Image-coloring/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connect to G Drive in order to retrieve the image datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Load all neccessary libraries 

In [ ]:

import keras
from tensorflow.keras.utils  import array_to_img, img_to_array, load_img
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.models import Sequential, Model
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
#OPTIMIZER
from tensorflow.keras.optimizers import RMSprop
import random

.... Add the Keras.Layers

In [ ]:
from keras.layers import Conv2D,BatchNormalization , ReLU , Conv2DTranspose

All image datasets loaded in numpy arrays

In [ ]:
Gray_DS = np.load("/content/drive/MyDrive/gray_scale.npy")
AB_1_NPY = np.load("/content/drive/MyDrive/ab1.npy")
AB_2_NPY = np.load("/content/drive/MyDrive/ab2.npy")
AB_3_NPY = np.load("/content/drive/MyDrive/ab3.npy")
AB_DS = np.concatenate([AB_1_NPY, AB_2_NPY, AB_3_NPY], axis=0)



Check type and size of arrays 

In [ ]:
print(type(Gray_DS))
print(type(AB_DS))

In [ ]:
print(Gray_DS.shape)
print(AB_DS.shape)

Pick a subset of the dataset to process due to limited resources

In [ ]:
start_split = 2000
end_split = 2500
end_non_seen= 3000
dataset_count = 500

Next I split the gray records in 2 set ( Train , Hidden ) because I want to keep records hidden by the algorithm. I will use them at the end for algorithm evaluation and prediction of colorized image

In [ ]:
X_Train = Gray_DS[start_split:end_split,:,:].astype("float32").reshape(dataset_count,Gray_DS.shape[1],Gray_DS.shape[2])

In [ ]:
X_Hidden = Gray_DS[end_split:end_non_seen,:,:].astype("float32").reshape(dataset_count,Gray_DS.shape[1],Gray_DS.shape[2])

In [ ]:
Y_Train = AB_DS[start_split:end_split,:,:].astype("float32")

In [ ]:
print(X_Train.shape)
print(X_Hidden.shape)
print(Y_Train.shape)

Function to convert the input image to row

In [ ]:
def conv_image_to_row(gray_img,splitting=dataset_count,inp_function=preprocess_input):
    Empty_imp = np.zeros((splitting,224,224,3))
    for indexing in range(0,3):
        Empty_imp[:splitting,:,:,indexing] = gray_img[:splitting]
        
    return inp_function(Empty_imp)

In [ ]:
Inp_Img = conv_image_to_row(X_Train,dataset_count)

Hidden_inp = conv_image_to_row(X_Hidden,dataset_count)



In [ ]:
print(Inp_Img.shape)
print(Hidden_inp.shape)

Function transform the input image by L*a*b color space again to RGB color space

In [ ]:
def tranf_lab_to_rgb(gray_img,ab_images,n=10):
    Empty_imp = np.zeros((n,224,224,3))
    
    Empty_imp[:,:,:,0] = gray_img[0:n:]
    Empty_imp[:,:,:,1:] = ab_images[0:n:]
    
    Empty_imp = Empty_imp.astype("uint8")
    
    Main_Img = []
    
    for indexing in range(0,n):
        Main_Img.append(cv2.cvtColor(Empty_imp[indexing],cv2.COLOR_LAB2RGB))
        
    Main_Img = np.array(Main_Img)
    
    return Main_Img

Based on documentation a grayscale image is near to L*A*B* format so we use the function COLOR_LAB2RGB to generate the colored image

In [ ]:
Output_img = preprocess_input(tranf_lab_to_rgb(X_Train,Y_Train,n=dataset_count))
print(Output_img.shape)

In [ ]:
Gray_emp_imp = np.zeros((dataset_count,Gray_DS.shape[1],Gray_DS.shape[2],1))
Gray_emp_imp[:,:,:,0] = X_Train
print(Gray_emp_imp.shape)

Setup CNN model's parameters

In [ ]:
compile_loss = "mse"
compile_optimizer = RMSprop(learning_rate=0.0001,decay=1e-8)
compile_metrics = ["accuracy"]
input_dim = (Inp_Img.shape[1],Inp_Img.shape[2],Inp_Img.shape[3])
output_class = 1

In [ ]:
Stopper = tf.keras.callbacks.EarlyStopping(monitor="loss",patience=3,mode="min")
Checkpoint = tf.keras.callbacks.ModelCheckpoint(monitor="val_accuracy",
                                                      save_best_only=True,
                                                      save_weights_only=True,
                                                      filepath="./modelcheck")

Setup the CNN layers

In [ ]:
Enc_Gray = Sequential()
Enc_Gray.add(Conv2D(32,(2,2),kernel_initializer = 'he_normal',padding = "same",use_bias = True))
Enc_Gray.add(BatchNormalization())
Enc_Gray.add(ReLU())

In [ ]:
Enc_Gray.add(Conv2D(64,(2,2),kernel_initializer = 'he_normal',padding = "same",use_bias = True))
Enc_Gray.add(BatchNormalization())
Enc_Gray.add(ReLU())
#

In [ ]:
Enc_Gray.add(Conv2D(128,(2,2),kernel_initializer = 'he_normal',padding = "same",use_bias = True))
Enc_Gray.add(BatchNormalization())
Enc_Gray.add(ReLU())


In [ ]:

Enc_Gray.add(Conv2D(256,(2,2),kernel_initializer = 'he_normal',padding = "same",use_bias = True))
Enc_Gray.add(BatchNormalization())
Enc_Gray.add(ReLU())

In [ ]:
Dec_Gray = Sequential()
Dec_Gray.add(Conv2DTranspose(128,(2,2),padding = "same",use_bias = True))
Dec_Gray.add(ReLU())
#

In [ ]:
Dec_Gray.add(Conv2DTranspose(64,(2,2),padding = "same",use_bias = True))
Dec_Gray.add(ReLU())
#

In [ ]:
Dec_Gray.add(Conv2DTranspose(32,(2,2),padding = "same",use_bias = True))
Dec_Gray.add(ReLU())
#

In [ ]:
Dec_Gray.add(Conv2DTranspose(3,(2,2),padding = "same",use_bias = True))
Dec_Gray.add(ReLU())

Execute the training of the model

In [ ]:

Exec_Encoder = Sequential([Enc_Gray,Dec_Gray])
Exec_Encoder.compile(loss=compile_loss,optimizer=compile_optimizer,metrics=compile_metrics)
Exec_Encoder_Model = Exec_Encoder.fit(Inp_Img,Output_img,epochs=25,callbacks=[Stopper,Checkpoint],batch_size=4)

Generate list of images by using the trained model for the known and the unknown dataset of images

In [ ]:
#PREDICTION

Predict_img = Exec_Encoder.predict(Inp_Img[0:100])
Predict_Hidden = Exec_Encoder.predict(Hidden_inp[0:100])


Evaluate the trained model with the hidden dataset 

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(20,20))
img_number =  random.randint(0,99)

Original_Img = Hidden_inp[img_number]
Predict_Image_AE = Predict_Hidden[img_number]
axis[0].imshow(Original_Img)
axis[0].set_xlabel(Original_Img.shape)
axis[0].set_ylabel(Original_Img.size)
axis[0].set_title("INPUT")
axis[1].imshow(Predict_Image_AE)
axis[1].set_xlabel(Predict_Image_AE.shape)
axis[1].set_ylabel(Predict_Image_AE.size)
axis[1].set_title("COLORIZED OUTPUT")


Test the trained model with the dataset used for its training 

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(20,20))
img_number =  random.randint(0,99)

Original_Img = Inp_Img[img_number]
Predict_Image_AE = Predict_img[img_number]
axis[0].imshow(Original_Img)
axis[0].set_xlabel(Original_Img.shape)
axis[0].set_ylabel(Original_Img.size)
axis[0].set_title("INPUT")
axis[1].imshow(Predict_Image_AE)
axis[1].set_xlabel(Predict_Image_AE.shape)
axis[1].set_ylabel(Predict_Image_AE.size)
axis[1].set_title("COLORIZED OUTPUT")